## Credits
Steam data - https://store.steampowered.com/search/?category1=998

## Download packages

In [11]:
#pip install selenium

## Imports

In [16]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import random
import re
from datetime import datetime

# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Save and load data from CSV

#### Save to CSV file

In [29]:
def save_to_csv(file_name, dataset):
    dataset.to_csv(file_name)

Save data before cleanup

In [19]:
save_to_csv("all_games_data.csv", df)

Save data after cleanup

In [37]:
save_to_csv("all_games_data_clean.csv", new_df)

#### Load dataset from CSV

In [38]:
def load_csv_from_project(file_name):
    return pd.read_csv(file_name)

Load data before cleanup

In [39]:
df = load_csv_from_project("all_games_data.csv").iloc[:,1:]
df.head()

Load data after cleanup

## Helper Functions

In [3]:
def write_soup_to_file(file_name : str, soup : BeautifulSoup):
    with open(file_name, "w", encoding='utf-8') as file:
        file.write(str(soup))

In [4]:
def load_soup_object(html_file_name : str):
    with open(html_file_name, "r", encoding='utf8') as file:
        soup = BeautifulSoup(file, "html.parser")
    return soup

In [13]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [14]:
def isint(num):
    try:
        int(num)
        return True
    except ValueError:
        return False

## Crawling With Selenium & BeautifulSoup

In [2]:
def get_driver_Firefox():
    return webdriver.Firefox()  

In [3]:
def infinite_scroll(driver : webdriver.firefox.webdriver.WebDriver):
    
    _isScrolling = True
    SCROLL_PAUSE_TIME = 0.8
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while _isScrolling:
                
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        sleep(SCROLL_PAUSE_TIME)
        SCROLL_PAUSE_TIME = random.uniform(0.8,3)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
                
        if new_height == last_height:
            #Another check
            sleep(20)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:            
                _isScrolling = False
                                
        last_height = new_height

In [35]:
def selenium_scrapping():
    url = "https://store.steampowered.com/search/?category1=998"
    driver = get_driver_Firefox()
    
    driver.get(url)
    driver.maximize_window()
    
    # Wait for the page to fully render before parsing it
    sleep(5)
    
    infinite_scroll(driver)
    
    print("Finish scroling... initialize BeautifulSoup")
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    #quits the driver instance, closing every associated window, which is opened.
    driver.quit()
    
#selenium_scrapping()

In [ ]:
#Saved soup for easier access.
#write_soup_to_file("All_Games.html", soup)

### Load BeautifulSoup

In [5]:
soup = load_soup_object("All_Games.html")

## Data Acquisition

In [7]:
def game_tag_exist(game , tag : str, classTag : str):
    if game.find(tag, attrs={"class",classTag}):
        return 1
    return 0

In [8]:
def game_get_text_tag(game, tag :str , classTag : str):
    return game.find(tag, attrs={"class", classTag}).get_text().strip()

In [9]:
def game_val_tag(game, tag :str , classTag : str):
    if game_tag_exist(game, tag, classTag):
        val = game_get_text_tag(game, tag, classTag)
        return np.nan if val == '' else val
        
    return np.nan

In [10]:
def game_prices(game, tag :str , classTag : str):
    prices = [np.nan]
    price_text = game_val_tag(game, "div", "search_price")
    if price_text is not np.nan:        
        if '₪' in price_text:
            prices = price_text.split('₪')
            prices.pop(0)
            if isfloat(prices[0]) is False:
                # price with ',' like 1,850.00
                if ',' in prices[0] :
                    prices[0] = float(prices[0].replace(',', ''))
                #Price like 49.00Free and discount 70% - is not relevent (only 2 like this)
                else:
                    prices = [np.nan]
            else:
                prices = [float(i) for i in prices]
        else:
            #free games
            prices = [float(0)]
            
    return prices
        

In [11]:
def scrape_games(soup : BeautifulSoup):
    game_title = list()
    game_win_support = list()
    game_mac_support = list()
    game_linux_support = list()    
    game_vr_only = list()
    game_vr_supported = list()
    game_relese_date = list()
    game_review = list()
    game_discount = list()
    game_price_discount = list()
    game_price = list()    
    
    all_games = soup("a",attrs={"class":"search_result_row"})    
    for game in all_games:
        game_title.append(game_val_tag(game, 'span', 'title'))
        game_win_support.append(game_tag_exist(game,'span', "win"))
        game_mac_support.append(game_tag_exist(game,'span', "mac"))
        game_linux_support.append(game_tag_exist(game,'span', "linux"))
        game_vr_only.append(game_tag_exist(game,'span', "vr_required"))
        game_vr_supported.append(game_tag_exist(game,'span', "vr_supported"))
        game_relese_date.append(game_val_tag(game, "div", "search_released"))
        game_review.append(game.find("span", attrs={"class",'search_review_summary'})['data-tooltip-html'] if game_tag_exist(game,'span', "search_review_summary") else np.nan)
        game_discount.append(game_val_tag(game, "div", "search_discount"))
        
        prices = game_prices(game, "div", "search_price")       
        game_price.append(prices[0])
        game_price_discount.append(prices[0] if len(prices) == 1 else prices[1])
         
    return pd.DataFrame({"title":game_title, "win_support":game_win_support
                         , "mac_support":game_mac_support, "linux_support":game_linux_support,
                         "vr_only":game_vr_only, "vr_supported":game_vr_supported,
                           "relese_date": game_relese_date, "review":game_review,
                         "discount":game_discount , "price_discount":game_price_discount ,"price":game_price })

In [15]:
df = scrape_games(soup)

## Handle Missing Values

In [35]:
#drop rows with missing price, release date and titles
df.dropna(subset=['price','relese_date','title'], inplace=True)
#no discount menas 0
df.discount.fillna(0, inplace=True)

## Data Cleaning

#### discount

In [31]:
def discount_clean(df : pd.DataFrame):
    df_copy = df.copy()
    df_copy['discount'] = [int(re.sub("[-%]" ,"", str(discount))) for discount in df_copy['discount']]
    return df_copy

#### review

In [32]:
def review_clean(df : pd.DataFrame):
    df_copy = df.copy()
    
    overall_review = list()
    percentage_positive_total_reviewed = list()
    total_reviewed = list()
    
    for rev in df_copy['review']:
        #For removing if else
        splited_data = ['no reviews','0 0 0 0']
        if rev is not np.nan:
            splited_data = rev.split('<br>')    
                
        reviewers = splited_data[1].split(' ')            
        overall_review.append(splited_data[0])        
        percentage_positive_total_reviewed.append(int(reviewers[0].replace('%','')))
        total_reviewed.append(int(reviewers[3].replace(',','')))
        
    df_copy['overall_review'] = overall_review
    df_copy['percentage_positive_total_reviewed'] = percentage_positive_total_reviewed
    df_copy['total_reviewed'] = total_reviewed
    
    df_copy.drop(['review'], axis = 1, inplace=True)
    
    return df_copy

#### relese_date

In [33]:
def relese_date_clean(df : pd.DataFrame):
    df_copy = df.copy()
    
    day = list()
    month = list()
    year = list()
    rows_to_remove = list()
    
    for idx in df_copy.relese_date.index.values:
        date_splited = df_copy.relese_date[idx].split(' ') 
        if len(date_splited) == 3 and isint(date_splited[0]):
            final_date = datetime.strptime(df_copy.relese_date[idx].replace(',',''), '%d %b %Y').strftime('%d %m %Y')
            date_splited = final_date.split(' ')
            day.append(date_splited[0])
            month.append(date_splited[1])
            year.append(date_splited[2])
        else:
            rows_to_remove.append(idx)
            
    df_copy.drop(index=rows_to_remove, inplace=True)
    df_copy.drop(['relese_date'], axis = 1, inplace=True)
    df_copy['day'] = day
    df_copy['month'] = month
    df_copy['year'] = year
    
    return df_copy

#### Cleaning Data

In [36]:
new_df = discount_clean(df)
new_df = review_clean(new_df)
new_df = relese_date_clean(new_df)

### Test on Data - Temporary

In [649]:
new_df[new_df["overall_review"] == "no reviews"]

,title,win_support,mac_support,linux_support,vr_only,vr_supported,discount,price_discount,price,overall_review,percentage_positive_total_reviewed,total_reviewed,day,month,year
227,EA Play,1,0,0,0,0,0,22.50,22.50,no reviews,0,0,31,08,2020
466,"Warhammer 40,000: Chaos Gate - Daemonhunters",1,0,0,0,0,0,169.95,169.95,no reviews,0,0,05,05,2022
569,The Elder Scrolls Online: High Isle,1,1,0,0,0,0,159.00,159.00,no reviews,0,0,06,06,2022
1630,S.T.A.L.K.E.R. 2: Heart of Chornobyl,1,0,0,0,0,0,239.00,239.00,no reviews,0,0,08,12,2022
2797,Forspoken,1,0,0,0,0,0,319.00,319.00,no reviews,0,0,11,10,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64200,Gator Parade,1,0,0,0,0,0,0.00,0.00,no reviews,0,0,11,04,2020
64202,The Delirium Vacation,1,0,0,0,0,0,10.95,10.95,no reviews,0,0,04,11,2019
64204,Juicy Army: Prologue,1,0,0,0,0,0,0.00,0.00,no reviews,0,0,15,12,2020
64205,Pixels can fight,1,0,0,0,0,0,18.50,18.50,no reviews,0,0,16,03,2021


In [650]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51445 entries, 0 to 64207
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   title                               51445 non-null  object 
 1   win_support                         51445 non-null  int64  
 2   mac_support                         51445 non-null  int64  
 3   linux_support                       51445 non-null  int64  
 4   vr_only                             51445 non-null  int64  
 5   vr_supported                        51445 non-null  int64  
 6   discount                            51445 non-null  int64  
 7   price_discount                      51445 non-null  float64
 8   price                               51445 non-null  float64
 9   overall_review                      51445 non-null  object 
 10  percentage_positive_total_reviewed  51445 non-null  int64  
 11  total_reviewed                      51445

In [651]:
comparison_column = np.where(new_df.price < new_df.price_discount, True, False)
print(comparison_column.any())

False


In [652]:
new_df.isnull().sum()

title                                 0
win_support                           0
mac_support                           0
linux_support                         0
vr_only                               0
vr_supported                          0
discount                              0
price_discount                        0
price                                 0
overall_review                        0
percentage_positive_total_reviewed    0
total_reviewed                        0
day                                   0
month                                 0
year                                  0
dtype: int64

In [451]:
all_games = soup.find_all("a", {"class": "search_result_row"})
all_games

[<a class="search_result_row ds_collapse_flag app_impression_tracked" data-ds-appid="730" data-ds-crtrids="[4]" data-ds-descids="[2,5]" data-ds-itemkey="App_730" data-ds-steam-deck-compat-handled="true" data-ds-tagids="[1663,1774,3859,3878,19,5711,5055]" data-gpnav="item" href="https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1" onmouseout="HideGameHover( this, event, 'global_hover' )" onmouseover="GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:730,&quot;public&quot;:1,&quot;v6&quot;:1} );">
 <div class="col search_capsule"><img src="https://cdn.akamai.steamstatic.com/steam/apps/730/capsule_sm_120.jpg?t=1641233427" srcset="https://cdn.akamai.steamstatic.com/steam/apps/730/capsule_sm_120.jpg?t=1641233427 1x, https://cdn.akamai.steamstatic.com/steam/apps/730/capsule_231x87.jpg?t=1641233427 2x"/></div>
 <div class="responsive_search_name_combined">
 <div class="col search_name ellipsis">
 <span class="title"

In [1]:
len(all_games)

NameError: name 'all_games' is not defined

# EDA

#### Scatter plot